In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import Levenshtein as Lv
import json

In [2]:
data = pd.read_csv('DS_Click_to_Product.csv',skiprows=2,low_memory=False)

In [7]:
data

,Account name,Customer ID,Campaign,Ad group,Final URL,Headline,Long headline,Headline 1,Headline 2,Headline 3,...,Avg. CPC,Converted currency code,Avg. CPC (Converted currency),Cost,Cost (Converted currency),Conversions,View-through conv.,Cost / conv.,Cost / conv. (Converted currency),Conv. rate
0,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,WV.1.1.10.1.5.00.1.0P.2_NF_NA_Health-Care-Plan,https://quotes.healthcare.com/health-insurance...,--,--,2021 Health Care Plans,See Plans in West Virginia,See Instant Rates Comparison,...,--,USD,--,0.0,--,0.0,0,--,--,0
1,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,AR.1.1.10.1.5.00.1.0P.2_NF_NA_Health-Plan,https://quotes.healthcare.com/health-plans/?SR...,--,--,Health Plans In Arkansas,Compare Rates Instantly,See Instant Rates Comparison,...,--,USD,--,0.0,--,0.0,0,--,--,0
2,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,HI.1.1.10.1.5.00.1.0P.2_NF_NA_Cheap-Health-Plan,https://quotes.healthcare.com/affordable-healt...,--,--,Compare Health Plans,View Hawaii Rates Now,See Instant Rates Comparison,...,--,USD,--,0.0,--,0.0,0,--,--,0
3,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,AR.1.1.10.1.5.00.1.0j.2_NF_NA_Health-Marketplace,https://quotes.healthcare.com/health-plans/?SR...,--,--,Health Marketplace Plans 2021,Plans Available Year-Round,See Instant Rates Comparison,...,--,USD,--,0.0,--,0.0,0,--,--,0
4,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3,carrier=allstate#product=accident_insurance,https://www.healthcare.com/?SRC=10,--,--,Accident Insurance,"Quotes Today, Coverage Tonight",HealthCare.com,...,--,USD,--,0.0,--,0.0,0,--,--,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460145,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,product=dental_insurance#product_descriptor=be...,https://www.healthcare.com/?SRC=10,--,--,Best Rated Dental Insurance,"{=COUNTDOWN(""2022/1/15 23:59:59"",""en-US"",20)} ...",Compare 2022 Plans,...,--,USD,--,0.0,--,0.0,0,--,--,0
460146,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,policy_detail=cost#product=health_insurance#pr...,https://www.healthcare.com/?SRC=10,--,--,2022 Affordable Care Act (ACA),"{=COUNTDOWN(""2022/1/15 23:59:59"",""en-US"",20)} ...",Cost in 2022,...,--,USD,--,0.0,--,0.0,0,--,--,0
460147,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,policy_detail=coverage#product=healthcare,https://www.healthcare.com/?SRC=10,--,--,Healthcare Coverage,"{=COUNTDOWN(""2021/12/15 23:59:59"",""en-US"",20)}...",2022 Plans from $29,...,--,USD,--,0.0,--,0.0,0,--,--,0
460148,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,KS.1.1.10.1.5.00.1.0g.2_NF_NA_Family-Health-In...,https://quotes.healthcare.com/family/?SRC=10&t...,--,--,Family Health Insurance 2020,Compare Kansas Rates,See Instant Rates Comparison,...,--,USD,--,0.0,--,0.0,0,--,--,0


In [9]:
data.dtypes

Account name                          object
Customer ID                           object
Campaign                              object
Ad group                              object
Final URL                             object
Headline                              object
Long headline                         object
Headline 1                            object
Headline 2                            object
Headline 3                            object
Headline 4                            object
Headline 5                            object
Headline 6                            object
Headline 7                            object
Headline 8                            object
Headline 9                            object
Headline 10                           object
Headline 11                           object
Headline 12                           object
Headline 13                           object
Headline 14                           object
Headline 15                           object
Descriptio

In [6]:
data['Account name'].value_counts()

HealthCare.com U65    424360
HealthCare.com O65     35790
Name: Account name, dtype: int64

In [19]:
data['Campaign'].sort_values().unique()

array(['O65_HC_DL_G_GDA_Display-Retargeting_AD_NT_NA',
       'O65_HC_G_ATLAS_Brand',
       'O65_HC_G_ATLAS_Competitors_RPA_Bucket:Misc',
       'O65_HC_G_ATLAS_Competitors_RPA_Bucket:Misc_weekend9.8',
       'O65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3',
       'O65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3_weekend9.8',
       'O65_HC_G_ATLAS_Dental_Vision',
       'O65_HC_G_ATLAS_Dental_Vision_weekend9.8',
       'O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid',
       'O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid_weekend9.8',
       'O65_HC_G_ATLAS_Med_Sup', 'O65_HC_G_ATLAS_Med_Sup_weekend9.8',
       'O65_HC_G_ATLAS_Medicaid_RPA_Bucket:Mid',
       'O65_HC_G_ATLAS_Medicaid_RPA_Bucket:Mid_weekend9.8',
       'O65_HC_G_ATLAS_Medicare_Advantage',
       'O65_HC_G_ATLAS_Medicare_Advantage_weekend9.8',
       'O65_HC_G_ATLAS_SVG', 'O65_HC_G_ATLAS_SVG_weekend9.8',
       'O65_HC_G_ATLAS_Top_Exact_Match',
       'O65_HC_G_ATLAS_Top_Exact_Match Top Exact Match (tROAS vs MCV)',
       'O65_HC_G_ATLAS_Top_Ex

In [21]:
data[data['Campaign']=='U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS vs mROAS']

,Account name,Customer ID,Campaign,Ad group,Final URL,Headline,Long headline,Headline 1,Headline 2,Headline 3,...,Avg. CPC,Converted currency code,Avg. CPC (Converted currency),Cost,Cost (Converted currency),Conversions,View-through conv.,Cost / conv.,Cost / conv. (Converted currency),Conv. rate
1401,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,place=missouri_us#product=insurance,https://www.healthcare.com/?SRC=10,--,--,2022 Insurance,Get Quotes for Quality Plans,HealthCare.com™ Official Site,...,--,USD,--,0.0,--,0.0,0,--,--,0
1402,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,place=virginia_us#product=health_insurance#pro...,https://www.healthcare.com/?SRC=10,--,--,Virginia Health Insurance,Find Low Cost Health Insurance,Get Instant Quotes This Year,...,--,USD,--,0.0,--,0.0,0,--,--,0
1403,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,product=health_insurance#product_descriptor=be...,https://www.healthcare.com/?SRC=10,--,--,Best Health Insurance,Enroll Now for Coverage,Get Instant Quotes This Year,...,--,USD,--,0.0,--,0.0,0,--,--,0
1404,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,policy_detail=lists#product=medical_insurance,https://www.healthcare.com/?SRC=10,--,--,Medical Insurance Lists,Plans Available Year-Round,2022 Plans from $29,...,--,USD,--,0.0,--,0.0,0,--,--,0
2436,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,place=michigan_us#program=cobra,https://www.healthcare.com/?SRC=10,--,--,2022 COBRA Michigan,COBRA Plans Available Now,Get Instant Quotes for 2022,...,--,USD,--,0.0,--,0.0,0,--,--,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274282,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,place=kentucky_us#product=health_insurance,https://www.healthcare.com/?SRC=10,--,--,Kentucky Health Insurance,"{=COUNTDOWN(""2022/1/15 23:59:59"",""en-US"",20)} ...",Extra 30 Days to Enroll,...,--,USD,--,0.0,--,0.0,0,--,--,0
275277,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,product=insurance#program_enrollment=open_enro...,https://www.healthcare.com/?SRC=10,--,--,2022 Insurance,HealthCare.com™ Official Site,99+ Plans from Top Carriers,...,--,USD,--,0.0,--,0.0,0,--,--,0
276176,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,place=south_carolina_us#product=government_hea...,https://www.healthcare.com/?SRC=10,--,--,SC Government Health Insurance,HealthCare.com™ Official Site,Coverage in South Carolina,...,--,USD,--,0.0,--,0.0,0,--,--,0
277863,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Generic_RPA_Bucket:High tROAS v...,insurance_period=temporary#place=ohio_us#produ...,https://www.healthcare.com/?SRC=10,--,--,Temporary Health Insurance,Ohio Health Insurance,"{=COUNTDOWN(""2021/12/15 23:59:59"",""en-US"",20)}...",...,--,USD,--,0.0,--,0.0,0,--,--,0


## Test 1

In [34]:
data['Cost'].dtypes

dtype('float64')

In [35]:
data[(data['Cost']==0.0) & (data['Conversions']==0.0)][['Campaign','Cost']]

,Campaign,Cost
0,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
1,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
2,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
3,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
4,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3,0.0
...,...,...
460145,U65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,0.0
460146,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,0.0
460147,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,0.0
460148,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0


In [36]:
t1 = data[(data['Cost']==0.0) & (data['Conversions']==0.0)][['Campaign','Cost']]

In [37]:
t1

,Campaign,Cost
0,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
1,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
2,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
3,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0
4,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3,0.0
...,...,...
460145,U65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,0.0
460146,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,0.0
460147,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,0.0
460148,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,0.0


In [38]:
t1['Campaign'].unique()

array(['U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB',
       'U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3',
       'U65_HC_SL_G_state-terms-roas2_AD_NT_EX',
       'O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid_weekend9.8',
       'U65_HC_G_ATLAS_Generic_RPA_Bucket:Mid',
       'U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier2',
       'U65_HC_G_ATLAS_Generic_Vision',
       'U65_HC_G_ATLAS_Generic_RPA_Bucket:High',
       'U65_HC_G_ATLAS_Generic_Dental',
       'O65_HC_G_ATLAS_Medicare_Advantage',
       'U65_HC_G_ATLAS_Generic_RPA_Bucket:Low',
       'U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier1',
       'O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid',
       'U65_HC_G_ATLAS_Competitors_RPA_Bucket:Vision',
       'U65_HC_G_ATLAS_Generic_Premium _TEST',
       'U65_HC_G_ATLAS_Competitors_RPA_Bucket:Catch_All',
       'O65_HC_G_ATLAS_Med_Sup', 'U65_HC_SL_G_RO_Freelancer_AD_NT_PB',
       'tROAS v tCPA U65_HC_SL_G_CW_marketplace-premium_AD_NT_EB',
       'U65_HC_SL_G_RO_Product-Type_AD_NT_EB > A/A',

In [39]:
t2 = data[(data['Cost']!=0.0) & (data['Conversions']!=0.0)][['Campaign','Cost']]

In [40]:
t2

,Campaign,Cost
346,U65_HC_G_ATLAS_Generic_RPA_Bucket:Low,20.52
3828,O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,208.61
4016,U65_HC_SL_G_RO_Freelancer_AD_NT_PB,5.82
5539,U65_HC_G_ATLAS_Brand,1.61
6005,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier1,37.27
...,...,...
458535,U65_HC_G_ATLAS_Generic_RPA_Bucket:Low,2.20
458769,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,2.24
458878,U65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,60.03
459186,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,16.69


In [42]:
# len(t2['Campaign'].unique())

29

### Test 2 - convert to json

In [49]:
# set up display area to show dataframe in jupyter qtconsole

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [55]:
data.head(20)

,Account name,Customer ID,Campaign,Ad group,Final URL,Headline,Long headline,Headline 1,Headline 2,Headline 3,Headline 4,Headline 5,Headline 6,Headline 7,Headline 8,Headline 9,Headline 10,Headline 11,Headline 12,Headline 13,Headline 14,Headline 15,Description,Description 1,Description 2,Description 3,Description 4,Business name,Call to action text,Display URL,Path 1,Path 2,Currency code,Clicks,Impr.,CTR,Avg. CPC,Converted currency code,Avg. CPC (Converted currency),Cost,Cost (Converted currency),Conversions,View-through conv.,Cost / conv.,Cost / conv. (Converted currency),Conv. rate
0,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,WV.1.1.10.1.5.00.1.0P.2_NF_NA_Health-Care-Plan,https://quotes.healthcare.com/health-insurance/?SRC=10&theme=health-plans&state=WV,--,--,2021 Health Care Plans,See Plans in West Virginia,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,WV Top Rated Health Insurance Plans Are Here. Find the Best Price & Save Today!,--,"Choose Your Deductible, Coverage, Co-Pays, and Get Customized Plan Recommendations.",--,--,--,--,--,Health_Plans,West_Virginia,USD,0,0,0,--,USD,--,0.0,--,0.0,0,--,--,0
1,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,AR.1.1.10.1.5.00.1.0P.2_NF_NA_Health-Plan,https://quotes.healthcare.com/health-plans/?SRC=10&theme=health-plans,--,--,Health Plans In Arkansas,Compare Rates Instantly,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,"2021 Health Plans are Available. Coverage Starting at Midnight, Care as Soon as Tomorrow.",--,"Choose Your Deductible, Coverage, Co-Pays, and Get Customized Plan Recommendations.",--,--,--,--,--,Health-Plan,Arkansas,USD,0,0,0,--,USD,--,0.0,--,0.0,0,--,--,0
2,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,HI.1.1.10.1.5.00.1.0P.2_NF_NA_Cheap-Health-Plan,https://quotes.healthcare.com/affordable-health-insurance/?SRC=10&theme=cheap&state=HI,--,--,Compare Health Plans,View Hawaii Rates Now,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,Get Low Prices From Just $53. Enter Your ZIP Code to Get Started!,--,"Choose Your Deductible, Coverage, Co-Pays, and Get Customized Plan Recommendations.",--,--,--,--,--,Health-Plans,Hawaii,USD,0,0,0,--,USD,--,0.0,--,0.0,0,--,--,0
3,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,AR.1.1.10.1.5.00.1.0j.2_NF_NA_Health-Marketplace,https://quotes.healthcare.com/health-plans/?SRC=10&theme=marketplace&state=AR,--,--,Health Marketplace Plans 2021,Plans Available Year-Round,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,Top Rated 2021 Medical Insurance Plans Are Here. Find The Best Plan & Save!,--,"Don't miss 2021 Open Enrollment! Only {=COUNTDOWN(""2020/12/16 00:00:00"",""en-US"",1)} left to get the best rates & stay covered.",--,--,--,--,--,Marketplace,Arkansas,USD,0,0,0,--,USD,--,0.0,--,0.0,0,--,--,0
4,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3,carrier=allstate#product=accident_insurance,https://www.healthcare.com/?SRC=10,--,--,Accident Insurance,"Quotes Today, Coverage Tonight",HealthCare.com,--,--,--,--,--,--,--,--,--,--,--,--,--,Low Cost Accident Insurance Plans Are Here. Find The Top Rated Plans This Year & Save!,Personalized Recommendations for Your Coverage. See if You Could Save on Insurance.,--,--,--,--,--,Allstate,--,USD,0,0,0,--,USD,--,0.0,--,0.0,0,--,--,0
5,HealthCare.com U65,476-322-7661,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3,carrier=anthem#place=illinois_us#product=health_insurance,https://www.healthcare.com/?SRC=10,--,--,Anthem Illinois Plans 2022,Illinois Health Insurance,2022 Health Insurance Plans,Record-Low Health Ins. Plans,Record-Low Health Ins. Costs,HealthCare.com™ Official Site,Find Health Insurance Plans,Stay Covered With Low Premiums,Coverage That Fits Your Needs,2022 Plans from $29,99+ Plans from Top Carriers,Get Instant Quotes for 2022,Compare 2022 Health Plans,

In [51]:
data.dtypes

Account name                          object
Customer ID                           object
Campaign                              object
Ad group                              object
Final URL                             object
Headline                              object
Long headline                         object
Headline 1                            object
Headline 2                            object
Headline 3                            object
Headline 4                            object
Headline 5                            object
Headline 6                            object
Headline 7                            object
Headline 8                            object
Headline 9                            object
Headline 10                           object
Headline 11                           object
Headline 12                           object
Headline 13                           object
Headline 14                           object
Headline 15                           object
Descriptio

In [53]:
pd.set_option('display.max_colwidth', -1)

/var/folders/5q/5s79sfds3w7_qbfv5xrwbj6r0000gn/T/ipykernel_73001/3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [54]:
data['Final URL'].head(1)

0    https://quotes.healthcare.com/health-insurance/?SRC=10&theme=health-plans&state=WV
Name: Final URL, dtype: object

In [56]:
json_df = data[['Account name','Campaign','Ad group','Final URL',
                'Headline','Long headline','Headline 1','Headline 2','Headline 3','Headline 4',
                'Headline 5','Headline 6','Headline 7','Headline 8','Headline 9','Headline 10',
                'Headline 11','Headline 12','Headline 13','Headline 14','Headline 15',
                'Description','Description 1','Description 2','Description 3','Description 4',
                'Business name','Call to action text','Display URL','Path 1','Path 2']]

In [64]:
json_df

,Account name,Campaign,Ad group,Final URL,Headline,Long headline,Headline 1,Headline 2,Headline 3,Headline 4,Headline 5,Headline 6,Headline 7,Headline 8,Headline 9,Headline 10,Headline 11,Headline 12,Headline 13,Headline 14,Headline 15,Description,Description 1,Description 2,Description 3,Description 4,Business name,Call to action text,Display URL,Path 1,Path 2
0,HealthCare.com U65,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,WV.1.1.10.1.5.00.1.0P.2_NF_NA_Health-Care-Plan,https://quotes.healthcare.com/health-insurance/?SRC=10&theme=health-plans&state=WV,--,--,2021 Health Care Plans,See Plans in West Virginia,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,WV Top Rated Health Insurance Plans Are Here. Find the Best Price & Save Today!,--,"Choose Your Deductible, Coverage, Co-Pays, and Get Customized Plan Recommendations.",--,--,--,--,--,Health_Plans,West_Virginia
1,HealthCare.com U65,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,AR.1.1.10.1.5.00.1.0P.2_NF_NA_Health-Plan,https://quotes.healthcare.com/health-plans/?SRC=10&theme=health-plans,--,--,Health Plans In Arkansas,Compare Rates Instantly,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,"2021 Health Plans are Available. Coverage Starting at Midnight, Care as Soon as Tomorrow.",--,"Choose Your Deductible, Coverage, Co-Pays, and Get Customized Plan Recommendations.",--,--,--,--,--,Health-Plan,Arkansas
2,HealthCare.com U65,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,HI.1.1.10.1.5.00.1.0P.2_NF_NA_Cheap-Health-Plan,https://quotes.healthcare.com/affordable-health-insurance/?SRC=10&theme=cheap&state=HI,--,--,Compare Health Plans,View Hawaii Rates Now,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,Get Low Prices From Just $53. Enter Your ZIP Code to Get Started!,--,"Choose Your Deductible, Coverage, Co-Pays, and Get Customized Plan Recommendations.",--,--,--,--,--,Health-Plans,Hawaii
3,HealthCare.com U65,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD_NT_EB,AR.1.1.10.1.5.00.1.0j.2_NF_NA_Health-Marketplace,https://quotes.healthcare.com/health-plans/?SRC=10&theme=marketplace&state=AR,--,--,Health Marketplace Plans 2021,Plans Available Year-Round,See Instant Rates Comparison,--,--,--,--,--,--,--,--,--,--,--,--,Top Rated 2021 Medical Insurance Plans Are Here. Find The Best Plan & Save!,--,"Don't miss 2021 Open Enrollment! Only {=COUNTDOWN(""2020/12/16 00:00:00"",""en-US"",1)} left to get the best rates & stay covered.",--,--,--,--,--,Marketplace,Arkansas
4,HealthCare.com U65,U65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3,carrier=allstate#product=accident_insurance,https://www.healthcare.com/?SRC=10,--,--,Accident Insurance,"Quotes Today, Coverage Tonight",HealthCare.com,--,--,--,--,--,--,--,--,--,--,--,--,--,Low Cost Accident Insurance Plans Are Here. Find The Top Rated Plans This Year & Save!,Personalized Recommendations for Your Coverage. See if You Could Save on Insurance.,--,--,--,--,--,Allstate,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460145,HealthCare.com U65,U65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,product=dental_insurance#product_descriptor=best_rated,https://www.healthcare.com/?SRC=10,--,--,Best Rated Dental Insurance,"{=COUNTDOWN(""2022/1/15 23:59:59"",""en-US"",20)} Left to Enroll",Compare 2022 Plans,--,--,--,--,--,--,--,--,--,--,--,--,"2021 Health Plans are Available. Coverage Starting at Midnight, Care as Soon as Tomorrow.",--,Best Rated Dental Insurance Plans Are Here. Find The Top Rated 2022 Plans & Save!,--,--,--,--,--,healthcare,DentalInsurance
460146,HealthCare.com U65,U65_HC_G_ATLAS_Generic_RPA_Bucket:High,policy_detail=cost#product=health_insurance#program=affordable_care_act_aca,https://www.healthcare.com/?SRC=10,--,--,2022 Affordable Care Act (ACA),"{=COUNTDOWN(""2022/1/15 23:59:59"",""en-US"",20)} Left to Enroll",Cost in 2022,--,--,--,--,--,--,--,--,--,--,--,--,"Open Enrollment Was Extended to January 15. {=

In [59]:
# result = json_df.to_json(orient="records")

In [66]:
# with open("result.json", "w") as outfile:
#     outfile.write(result)

In [65]:
# print(result)

## Test 3

In [67]:
group = json_df.groupby('Account name')

df2 = group.apply(lambda x: x['Campaign'].unique())



In [70]:
df2

Account name
HealthCare.com O65    [O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid_weekend9.8, O65_HC_G_ATLAS_Medicare_Advantage, O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid, O65_HC_G_ATLAS_Med_Sup, tROAS - O65_HC_SL_G_MO_premium-medicare_AD_NT_EB, tCPA vs. tROAS O65_HC_SL_G_CW_generic-medicare_AD_NT_EB, tCPA vs. tROAS O65_HC_SL_G_CW_premium-medicare_AD_NT_EB, O65_HC_DL_G_GDA_Display-Retargeting_AD_NT_NA, O65_HC_G_ATLAS_Dental_Vision_weekend9.8, O65_HC_G_ATLAS_Dental_Vision, O65_HC_G_ATLAS_Top_Exact_Match Top Exact Match (tROAS vs MCV), TCPA VS. TROAS - O65_HC_SL_G_CP_premium-medicare_AD_NT_EB, tROAS - O65_HC_SL_G_MO_carrier-generic_AD_NT_EB, O65_HC_G_ATLAS_Competitors_RPA_Bucket:Misc_weekend9.8, O65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3, O65_HC_G_ATLAS_Medicaid_RPA_Bucket:Mid, O65_HC_G_ATLAS_Medicaid_RPA_Bucket:Mid_weekend9.8, O65_HC_G_ATLAS_Competitors_RPA_Bucket:Misc, O65_HC_G_ATLAS_Medicare_Advantage_weekend9.8, O65_HC_G_ATLAS_Med_Sup_weekend9.8, tROAS - O65_HC_SL_G_MO_generic-medicare_AD_NT_EB

In [71]:
df2.to_dict()

{'HealthCare.com O65': array(['O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid_weekend9.8',
        'O65_HC_G_ATLAS_Medicare_Advantage',
        'O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid', 'O65_HC_G_ATLAS_Med_Sup',
        'tROAS - O65_HC_SL_G_MO_premium-medicare_AD_NT_EB',
        'tCPA vs. tROAS O65_HC_SL_G_CW_generic-medicare_AD_NT_EB',
        'tCPA vs. tROAS O65_HC_SL_G_CW_premium-medicare_AD_NT_EB',
        'O65_HC_DL_G_GDA_Display-Retargeting_AD_NT_NA',
        'O65_HC_G_ATLAS_Dental_Vision_weekend9.8',
        'O65_HC_G_ATLAS_Dental_Vision',
        'O65_HC_G_ATLAS_Top_Exact_Match Top Exact Match (tROAS vs MCV)',
        'TCPA VS. TROAS - O65_HC_SL_G_CP_premium-medicare_AD_NT_EB',
        'tROAS - O65_HC_SL_G_MO_carrier-generic_AD_NT_EB',
        'O65_HC_G_ATLAS_Competitors_RPA_Bucket:Misc_weekend9.8',
        'O65_HC_G_ATLAS_Competitors_RPA_Bucket:Tier3',
        'O65_HC_G_ATLAS_Medicaid_RPA_Bucket:Mid',
        'O65_HC_G_ATLAS_Medicaid_RPA_Bucket:Mid_weekend9.8',
        'O65_HC_G_ATLAS_C

In [73]:
# df3 = df2.apply(lambda x: x['Ad group'].unique())

## Test 4

In [74]:
group4 = json_df.groupby(['Account name','Campaign','Ad group'])

In [84]:
# json_df[group4]

In [76]:
print(group4)

In [88]:
df4 = group4.apply(lambda x: x['Final URL'].unique())

In [89]:
df4

Account name        Campaign                                                  Ad group                                            
HealthCare.com O65  O65_HC_DL_G_GDA_Display-Retargeting_AD_NT_NA              O65_All_Visitors                                        [https://quotes.healthcare.com/special-enrollment-period-aca-8-15/?SRC=120R&theme=special-enrollment, https://medicare.healthcare.com/medicare-plan-g-vs-plan-n-v2/?SRC=120R&theme=plan-gvsplan-n, https://medicare.healthcare.com/medicare-secrets-map/?SRC=120R&theme=secret-lp, https://medicare.healthcare.com/medicare-part-b/?SRC=120R&state=&theme=part_b, https://medicare.healthcare.com/medicare-open-enrollment-2022-general/?SRC=120R&theme=aep, https://medicare.healthcare.com/medicare/plan-g/?SRC=120R&theme=plan-g, https://medicare.healthcare.com/seniors-save-3/?SRC=120R&theme=plan-g, https://medicare.healthcare.com/medicare-part-b-3gb/?SRC=120R&theme=part-b, https://medicare.healthcare.com/medicare-advantage/?SRC=120R&theme=medi

In [ ]:
Headline	Long headline	Headline 1	Headline 2	Headline 3	Headline 4	Headline 5	Headline 6	Headline 7	Headline 8	Headline 9	Headline 10	Headline 11	Headline 12	Headline 13	Headline 14	Headline 15	Description	Description 1	Description 2	Description 3	Description 4	

In [97]:
df5 = group4.apply(lambda x: x['Headline 1'].unique())

In [98]:
df5

Account name        Campaign                                                  Ad group                                            
HealthCare.com O65  O65_HC_DL_G_GDA_Display-Retargeting_AD_NT_NA              O65_All_Visitors                                        [ --]                                                                                           
                                                                              O65_All_Visitors_Excluding_All_Converters               [ --]                                                                                           
                                                                              O65_CustomerMatch_Audience                              [ --]                                                                                           
                                                                              O65_Engaged_Audience_Excluding_All_Converters           [ --]                                     

In [1]:
# df5.to_dict()